##### Imports


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from importlib import reload
import sys

##### Download data


In [2]:
!gdown "https://drive.google.com/uc?id=1bGchkm9ilJllheYPbSttuScrcy5_gYLd" -O ../../data/2023-citibike-tripdata.zip

Downloading...
From (original): https://drive.google.com/uc?id=1bGchkm9ilJllheYPbSttuScrcy5_gYLd
From (redirected): https://drive.google.com/uc?id=1bGchkm9ilJllheYPbSttuScrcy5_gYLd&confirm=t&uuid=caa5b04b-4546-4c91-be94-edd39791df28
To: /Users/humbertoyusta/classes/buffer-sharing-in-multi-tenant-db/data/2023-citibike-tripdata.zip
100%|████████████████████████████████████████| 465M/465M [01:41<00:00, 4.57MB/s]


##### Extracting data


In [4]:
zip_path = "../../data/2023-citibike-tripdata.zip"
output_path = "../../data/"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(output_path)

##### File names of data files

There is one file per month.


In [2]:
FILE_NAMES = [
    "202301-citibike-tripdata.csv",
    "202302-citibike-tripdata.csv",
    "202303-citibike-tripdata.csv",
    "202304-citibike-tripdata.csv",
    "202305-citibike-tripdata.csv",
    "202306-citibike-tripdata.csv",
    "202307-citibike-tripdata.csv",
    "202308-citibike-tripdata.csv",
    "202309-citibike-tripdata.csv",
    "202310-citibike-tripdata.csv",
    "202311-citibike-tripdata.csv",
    "202312-citibike-tripdata.csv",
]

#### First experiment test selection

Tenants are chosen based on if the bike was classic or electric, and if it was casual or usual member (therefore 4 tenants)

Page ids are chosen based on tuple (start_station_id, end_station_id)


In [69]:
file_names = FILE_NAMES[0:4]

for index, file_name in enumerate(file_names):
    df = pd.read_csv(
        output_path + "2023-citibike-tripdata/" + file_name,
        dtype={"start_station_id": str, "end_station_id": str},
    )

    df.dropna(subset=["start_station_id", "end_station_id"], inplace=True)

    df.sort_values(by="started_at", inplace=True)

    df = df[df["rideable_type"] != "docked_bike"]

    df["tenant_id"] = (
        (df["rideable_type"].rank(method="dense") - 1) * df["member_casual"].nunique()
        + (df["member_casual"].rank(method="dense") - 1)
    ).astype(int) + 1

    df["stations_id_concat"] = df["start_station_id"] + "_" + df["end_station_id"]

    df["page_id"] = (
        df.groupby("tenant_id")["stations_id_concat"].rank(method="dense").astype(int)
    )

    selected_df = df[
        [
            "tenant_id",
            "page_id",
        ]
    ]

    selected_df.to_csv(
        output_path + f"citibike_exp_1_case_{index + 1}.csv",
        index=False,
    )

#### Second experiment test selection

Tenants are chosen based on if the bike was classic or electric, and if it was casual or usual member (therefore 4 tenants)

Page ids are chosen based on tuple (rank(start_station_id) // 5, rank(end_station_id) // 5)

Division by 5 is added to reduce total number of different pages per tenant.


In [7]:
file_names = FILE_NAMES[4:8]

for index, file_name in enumerate(file_names):
    df = pd.read_csv(
        output_path + "2023-citibike-tripdata/" + file_name,
        dtype={"start_station_id": str, "end_station_id": str},
    )

    df.dropna(subset=["start_station_id", "end_station_id"], inplace=True)

    df.sort_values(by="started_at", inplace=True)

    df = df[df["rideable_type"] != "docked_bike"]

    df["tenant_id"] = (
        (df["rideable_type"].rank(method="dense") - 1) * df["member_casual"].nunique()
        + (df["member_casual"].rank(method="dense") - 1)
    ).astype(int) + 1

    df["start_station_id_number"] = (
        df["start_station_id"].rank(method="dense").astype(int) // 5
    )
    df["end_station_id_number"] = (
        df["end_station_id"].rank(method="dense").astype(int) // 5
    )

    df["stations_id_concat"] = (
        df["start_station_id_number"].astype(str)
        + "_"
        + df["end_station_id_number"].astype(str)
    )

    df["page_id"] = (
        df.groupby("tenant_id")["stations_id_concat"].rank(method="dense").astype(int)
    )

    selected_df = df[
        [
            "tenant_id",
            "page_id",
        ]
    ]

    selected_df = selected_df.head(500000).copy()

    selected_df["page_id"] = (
        selected_df.groupby("tenant_id")["page_id"].rank(method="dense").astype(int)
    )

    selected_df.to_csv(
        output_path + f"citibike_exp_2_case_{index + 1}.csv",
        index=False,
    )

#### Third experiment test selection

Tenants are chosen based on if the bike was classic or electric, and if it was casual or usual member (therefore 4 tenants)

Page ids are chosen based on start station id.


In [8]:
file_names = FILE_NAMES[8:12]

for index, file_name in enumerate(file_names):
    df = pd.read_csv(
        output_path + "2023-citibike-tripdata/" + file_name,
        dtype={"start_station_id": str, "end_station_id": str},
    )

    df.dropna(subset=["start_station_id", "end_station_id"], inplace=True)

    df.sort_values(by="started_at", inplace=True)

    df = df[df["rideable_type"] != "docked_bike"]

    df["tenant_id"] = (
        (df["rideable_type"].rank(method="dense") - 1) * df["member_casual"].nunique()
        + (df["member_casual"].rank(method="dense") - 1)
    ).astype(int) + 1

    df["page_id"] = (
        df.groupby("tenant_id")["start_station_id"].rank(method="dense").astype(int)
    )

    selected_df = df[
        [
            "tenant_id",
            "page_id",
        ]
    ]

    selected_df = selected_df.head(50000).copy()

    selected_df["page_id"] = (
        selected_df.groupby("tenant_id")["page_id"].rank(method="dense").astype(int)
    )

    selected_df.to_csv(
        output_path + f"citibike_exp_3_case_{index + 1}.csv",
        index=False,
    )